In [1]:
import requests
import pandas as pd
from datetime import datetime
import json
import os

In [2]:
base_url = "https://api.energidataservice.dk/dataset/"

In [12]:
dataset = 'AfrrReservesNordic'
start_date = '2023-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

print(f"Using start date: {start_date}")
print(f"Using end date: {end_date}")

filters = {'PriceArea': ['DK2']}
output_filename = 'afrr_reserves_dk2_2023_today.csv'
data_directory = '../data'

Using start date: 2023-01-01
Using end date: 2025-04-02


In [6]:
def fetch_data(dataset_name, start, end, limit=10000, filters=None):
    """Fetches data from Energinet API with pagination handling."""
    all_records = []
    offset = 0
    print(f"Fetching data for {dataset_name} from {start} to {end}...")
    while True:
        processed_filters = {}
        if filters:
            for key, value in filters.items():
                if not isinstance(value, list):
                    processed_filters[key] = [value]
                else:
                    processed_filters[key] = value

        params = {
            'offset': offset,
            'limit': limit,
            'start': start,
            'end': end,
            'filter': json.dumps(processed_filters) if processed_filters else None,
            'sort': 'HourUTC ASC'
        }
        response = None
        try:
            response = requests.get(base_url + dataset_name, params=params)
            response.raise_for_status()
            data = response.json()
            records = data.get('records', [])
            all_records.extend(records)

            print(f"Fetched {len(records)} records (offset {offset}). Total fetched so far: {len(all_records)}")

            # Check if the total number of records has been reached or no more records are returned
            total_expected = data.get('total')
            if not records or len(records) < limit or (total_expected is not None and len(all_records) >= total_expected):
                 print(f"Finished fetching {dataset_name}. Total records according to API: {data.get('total', 'N/A')}. Fetched records: {len(all_records)}")
                 break

            offset += len(records) # Increment offset by the number of records actually fetched

        except requests.exceptions.HTTPError as http_err:
             print(f"HTTP error occurred while fetching data for {dataset_name}: {http_err}")
             if response is not None:
                 print(f"URL attempted: {response.url}")
                 print(f"Response status: {response.status_code}")
                 print(f"Response text: {response.text[:500]}...")
             break # Stop fetching on HTTP error
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {dataset_name}: {e}")
            if response is not None:
                 print(f"URL attempted: {response.url}")
            break # Stop fetching on other request errors
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            break # Stop fetching on unexpected errors

    # Convert to DataFrame after fetching all records
    if not all_records:
        print(f"Warning: No records fetched for {dataset_name}. Returning empty DataFrame.")
        return pd.DataFrame()
    else:
        return pd.DataFrame(all_records)

In [13]:
afrr_df = fetch_data(dataset_name=dataset, start=start_date, end=end_date, filters=filters)

Fetching data for AfrrReservesNordic from 2023-01-01 to 2025-04-02...
Fetched 10000 records (offset 0). Total fetched so far: 10000
Fetched 9727 records (offset 10000). Total fetched so far: 19727
Finished fetching AfrrReservesNordic. Total records according to API: 19727. Fetched records: 19727


In [14]:
output_filepath = os.path.join(data_directory, output_filename)
afrr_df.to_csv(output_filepath, index=False)